In [1]:
import ibm_db

In [2]:
#Replace the placeholder values with the actuals for your Db2 Service Credentials
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_hostname = "dashdb-txn-sbox-yp-lon02-06.services.eu-gb.bluemix.net"
dsn_port = "50000"                    # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_uid = "vqt95830"                 # e.g. "abc12345"
dsn_pwd = "9g5jhtxvvx^qb1n8"                 # e.g. "7dBZ3wWt9XN6$o0J"

In [4]:
# Create database connection
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )


Connected to database:  BLUDB as user:  vqt95830 on host:  dashdb-txn-sbox-yp-lon02-06.services.eu-gb.bluemix.net


In [5]:
#Lets first drop the table INSTRUCTOR in case it exists from a previous attempt

dropQuery = "drop table INSTRUCTOR"

In [6]:
# Execute the drop statement

dropStmt = ibm_db.exec_immediate(conn, dropQuery)

Exception: [IBM][CLI Driver][DB2/LINUXX8664] SQL0204N  "VQT95830.INSTRUCTOR" is an undefined name.  SQLSTATE=42704 SQLCODE=-204

In [9]:
#Construct the Create Table DDL statement
createQuery = "create table INSTRUCTOR(id INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2) )"

In [10]:
#Now fill in the name of the method and execute the statement
createStmt = ibm_db.exec_immediate(conn, createQuery)

In [14]:
insertQuery = "insert into INSTRUCTOR values (1, 'Rav', 'Ahuja', 'TORONTO', 'CA')"
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

In [16]:
insertQuery2 = "insert into INSTRUCTOR values (2, 'Raul', 'Chong', 'MARKHAM', 'CA'),(3, 'Hima', 'Vasudevan', 'CHICAGO', 'US')"
inssertStmt = ibm_db.exec_immediate(conn, insertQuery2)

In [17]:
#Construct the query that retrieves all rows from the INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#Fetch the Dictionary (for the first row only)
ibm_db.fetch_both(selectStmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA',
 4: 'CA'}

In [18]:
#Fetch the rest of the rows and print the ID and FNAME for those rows
while ibm_db.fetch_row(selectStmt) != False:
   print (" ID:",  ibm_db.result(selectStmt, 0), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))

 ID: 2  FNAME: Raul
 ID: 3  FNAME: Hima


In [21]:
updateQuery = "update INSTRUCTOR set CITY='MOOSETOWN' where FNAME='Rav'"

updateStmt = ibm_db.exec_immediate(conn, updateQuery)

In [22]:
import pandas

In [23]:
import ibm_db_dbi

In [24]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [25]:
#query statement to retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#retrieve the query results into a pandas dataframe
pdf = pandas.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame
pdf.LNAME[0]

'Ahuja'

In [26]:
#print the entire data frame
pdf

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,MARKHAM,CA
2,3,Hima,Vasudevan,CHICAGO,US


In [27]:
pdf.shape

(3, 5)

In [28]:
ibm_db.close(conn)

True